In [29]:

import time

In [5]:
import requests
import json
# import pandas as pd
# from datetime import datetime

# API Configuration
BASE_URL = "http://localhost:8001"
HEADERS = {"Content-Type": "application/json"}

def post_request(endpoint: str, payload: dict, headers: dict, timeout: int = 15):
    url = f"{BASE_URL}/{endpoint}"
    
    try:
        response = requests.post(
            url,
            json=payload,
            headers=headers,
            timeout=timeout
        )
        
        print(f"\n📡 POST to {endpoint}")
        print(f"Status Code: {response.status_code}")
        print(f"Response Time: {response.elapsed.total_seconds():.2f}s")
        
        if response.status_code == 200:
            result = response.json()
            print("✅ Request successful!")
            return result
        else:
            print(f"❌ Request failed: {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Request error: {e}")
        return None

## Product matching (With Image)

In [ ]:
def product_matching(caption_payload, user_messages):
    start = time.time()

    ## Image Captioning request
    start = time.time()
    if caption_payload["image_path"] == "":
        print("No image provided for captioning.")
        image_caption = "No image provided for captioning."
    else:
        print("image captioning...")
        result = post_request("image_captioning", caption_payload, HEADERS)
        image_caption = result['results']

    print("find docs...")
    doc_payload = {
        "user_query": f'''{user_messages}''',
        "image_prompt": image_caption
    }

    result = post_request("finding_documents", doc_payload, HEADERS, timeout=30)
    reference = result['results']
    print(reference)


    print("generate response...")
    payload = {
        "question": f'''{user_messages}''',
        "reference": f'''{reference}'''
    }

    result = post_request("generation", payload, HEADERS, timeout=30)
    print(result["response"])

    end = time.time()
    print(f"Total time taken: {end - start:.2f} seconds")
    return result["response"], reference

In [43]:
## Setting input
caption_payload = {
    "image_path": "Data/Image12.png"
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }

product_matching(caption_payload, user_messages)

image captioning...

📡 POST to image_captioning
Status Code: 200
Response Time: 3.53s
✅ Request successful!
find docs...

📡 POST to finding_documents
Status Code: 200
Response Time: 9.00s
✅ Request successful!
[{'search_term': 'floral maxi dress', 'search_results': [{'score': 0.7890574, 'id': 'Image40.jpg', 'name': 'Yellow Floral Spaghetti Strap Midi Dress', 'description': 'This enchanting yellow midi dress features delicate spaghetti straps and a flattering sweetheart neckline with a charming ruched or tie-front detail at the bust. Adorned with a vibrant floral print in shades of pink, purple, and orange, the dress has a flowing silhouette that gracefully falls to a midi length with an asymmetrical hem. Style Vibe: Romantic Summer / Whimsical Chic. Fit: Fitted bust with a flowing A-line midi skirt. Material: Lightweight, airy fabric (likely chiffon or georgette) with a subtle sheen. Care: Dry clean recommended or delicate hand wash. Matches Well With: Outerwear: A light white cardigan

('Based on the product information provided, here are some recommendations for your summer occasions:\n\nFor the Beach Party (Casual & Cute):\n1. Yellow Floral Spaghetti Strap Midi Dress ($98.00)\n   - Vibrant floral print in pink, purple, and orange\n   - Lightweight, airy fabric\n   - Romantic summer style\n   - Matches well with strappy sandals or flat sandals\n\n2. Navy V-Neck Tiered Maxi Dress ($85.00)\n   - Relaxed, flowing silhouette\n   - Ideal for warm weather\n   - Resort chic style\n   - Pairs well with flat sandals or espadrille wedges\n\nFor the Formal Dinner:\n1. Dusty Rose Pleated Midi Dress ($95.00)\n   - Elegant, vintage feminine style\n   - Fitted bodice with cinched waist\n   - Structured polyester blend fabric\n   - Matches well with pointed-toe heels or block heels\n\n2. Zara Sequin Maxi Skirt ($129.90)\n   - Glamorous evening elegance\n   - Black sequin design\n   - Can be paired with a silk camisole or long-sleeve top\n\nWould you like more details about any of t

In [ ]:
## Setting input
caption_payload = {
    "image_path": ""
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }
product_matching(caption_payload, user_messages)

### Product Matching (Without Image)

In [ ]:
## Setting input
caption_payload = {
    "image_path": ""
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }

## Style matching

In [38]:
def style_matching(caption_payload, user_messages):
    start = time.time()
    if caption_payload["image_path"] == "":
        print("No image provided for captioning.")
        image_caption = "No image provided for captioning."
    else:
        print("image captioning...")
        result = post_request("image_captioning", caption_payload, HEADERS)
        image_caption = result['results']

    print("style complement...")
    doc_payload = {
        "user_query": f'''{user_messages}''',
        "image_prompt": image_caption
    }
    result = post_request("style_complement", doc_payload, HEADERS, timeout=30)
    style_complement = result['results']
    print("find docs...")
    doc_payload = {
        "user_query": style_complement,
        "image_prompt": image_caption
    }

    result = post_request("finding_documents", doc_payload, HEADERS, timeout=30)
    reference = result['results']
    print(reference)


    print("generate response...")
    payload = {
        "question": f'''{user_messages}''',
        "reference": f'''{reference}'''
    }

    result = post_request("generation", payload, HEADERS, timeout=30)
    print(result["response"])

    end = time.time()
    print(f"Total time taken: {end - start:.2f} seconds")
    return result["response"], reference
        

In [39]:
## Setting input
caption_payload = {
    "image_path": "Data/Image12.png"
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }

In [40]:
style_matching(caption_payload, user_messages)

image captioning...

📡 POST to image_captioning
Status Code: 200
Response Time: 2.92s
✅ Request successful!
style complement...

📡 POST to style_complement
Status Code: 200
Response Time: 4.79s
✅ Request successful!
find docs...

📡 POST to finding_documents
Status Code: 200
Response Time: 20.52s
✅ Request successful!
[{'search_term': 'professional button-down midi dresses', 'search_results': [{'score': 0.7528786, 'id': 'Image17.png', 'name': 'Dusty Rose Pleated Midi Dress', 'description': 'This elegant dusty rose midi dress features a classic collared neckline with a subtle button placket, and charming short sleeves. The fitted bodice cinches at the waist with a subtle belt detail, flowing into a beautifully pleated A-line skirt that creates a graceful silhouette. This dress combines timeless charm with modern sophistication. Style Vibe: Vintage Feminine / Classic Elegance. Fit: Fitted bodice with a cinched waist, flowing into a pleated A-line midi skirt. Material: Smooth, structured p

("Based on the provided product information, I can recommend the Yellow Floral Spaghetti Strap Midi Dress for a beach party or summer event. Here are the details:\n\n- Name: Yellow Floral Spaghetti Strap Midi Dress\n- Price: $98.00\n- Description: This enchanting yellow midi dress features delicate spaghetti straps and a flattering sweetheart neckline. It's adorned with a vibrant floral print in shades of pink, purple, and orange.\n- Style Vibe: Romantic Summer / Whimsical Chic\n- Fit: Fitted bust with a flowing A-line midi skirt\n- Matches Well With:\n  - Outerwear: A light white cardigan or a cropped denim jacket\n  - Shoes: Strappy heeled sandals, elegant mules, or dressy flat sandals\n  - Accessories: A delicate clutch bag or a floral-embellished hair clip\n\nFor a formal dinner, I recommend the Dusty Rose Pleated Midi Dress:\n\n- Name: Dusty Rose Pleated Midi Dress\n- Price: $95.00\n- Description: An elegant dusty rose midi dress with a classic collared neckline, button placket, a

In [41]:
## Setting input
caption_payload = {
    "image_path": ""
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }

style_matching(caption_payload, user_messages)

No image provided for captioning.
style complement...

📡 POST to style_complement
Status Code: 200
Response Time: 2.46s
✅ Request successful!
find docs...

📡 POST to finding_documents
Status Code: 200
Response Time: 17.44s
✅ Request successful!
[{'search_term': 'casual cute beach party dress', 'search_results': [{'score': 0.69277227, 'id': 'Image29.png', 'name': 'Mustard Yellow Swing Mini Dress', 'description': 'This cheerful mustard yellow mini dress features a simple crew neck and short sleeves, with a comfortable and flattering swing silhouette that flares gently from the bodice. Its vibrant color and effortless design make it a perfect casual piece for bright, sunny days, offering comfort without compromising on style. Style Vibe: Casual & Bright / Playful Everyday. Fit: Relaxed, A-line swing fit. Material: Soft, stretchy jersey knit (cotton blend). Care: Machine wash cold. Matches Well With: Outerwear: A light denim jacket or a white casual cardigan. Shoes: White sneakers, flat sa

("Based on the product information provided, here are some recommendations for your summer outfits:\n\nFor the Beach Party:\n1. Mustard Yellow Swing Mini Dress ($45.00) - A casual and bright dress with a comfortable swing silhouette. It's perfect for a beach party and can be paired with white sneakers or flat sandals.\n\n2. Blue Tropical Print Sarong Skirt ($45.00) - A vibrant sarong skirt with a beachy vibe that can be styled with a white bikini top or a lightweight camisole.\n\nFor the Formal Dinner:\n1. Dusty Rose Pleated Midi Dress ($95.00) - An elegant dress with a collared neckline, fitted bodice, and a subtle belt detail. It's perfect for a formal dinner and can be paired with pointed-toe heels or ballet flats.\n\n2. Yellow Floral Spaghetti Strap Midi Dress ($98.00) - A romantic midi dress with a sweetheart neckline and floral print, suitable for a semi-formal dinner. It can be accessorized with strappy heeled sandals and a delicate clutch.\n\nWould you like more details about a